In [24]:
from os.path import join, basename, splitext
from glob import glob
from dask import dataframe as dd
from matplotlib import rcParams
import pandas as pd
import dask
from collections import Counter
import pickle
import numpy as np
from datetime import datetime
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, r2_score

from deep_aqi import ROOT


pd.set_option('expand_frame_repr', False)
pd.set_option('max_columns', 50)
pd.set_option('max_rows', 25)

In [2]:
PROCESSED_DATA = join(ROOT, 'data', 'processed')
INTERIM_DATA = join(ROOT, 'data', 'interim')
RAW_DATA = join(ROOT, 'data', 'raw')

In [13]:
pm25_path = join(PROCESSED_DATA, '88101.parquet')
pm25 = pd.read_parquet(pm25_path)

In [4]:
pm25.head()

,SiteCode,LocalDate,WindDir,WindSpeed,Temperature,Pressure,RelHum,PM2.5 - Local Conditions
index,,,,,,,,
0,California_Fresno_5001.0,2014-01-01 00:00:00,63.0,1.749611,5.555556,100660.0,67.0,80.0
1,California_Fresno_5001.0,2014-01-01 05:00:00,71.0,3.304821,3.888889,100660.0,69.0,43.0
2,California_Fresno_5001.0,2014-01-01 15:00:00,280.0,5.054432,17.777778,100520.0,29.0,62.0
3,California_Fresno_5001.0,2014-01-01 19:00:00,50.0,1.749611,11.666667,100660.0,53.0,83.0
4,California_Fresno_5001.0,2014-01-01 21:00:00,67.0,3.304821,8.888889,100660.0,58.0,75.0


In [50]:
to_concat = []
for name, df in pm25.groupby(by='SiteCode'):
    df = df.sort_values(['SiteCode', 'LocalDate'])
    df.set_index('LocalDate', inplace=True)
    
    timeshift = df.iloc[:, -1].reset_index()

    timeshift['PredictTime'] = timeshift.LocalDate
    timeshift.PredictTime = timeshift.PredictTime + pd.Timedelta('1D')
    timeshift = timeshift.rename(columns={'PM2.5 - Local Conditions': 'PM2.5-24h'})
    
    timeshift = timeshift[['PredictTime', 'PM2.5-24h']]
    df.reset_index(inplace=True)
    df = pd.merge(df, timeshift, left_on='LocalDate', right_on='PredictTime', how='inner')
    
    mae = mean_absolute_error(df['PM2.5 - Local Conditions'], df['PM2.5-24h'])
    mse = mean_squared_error(df['PM2.5 - Local Conditions'], df['PM2.5-24h'])
    mdae = median_absolute_error(df['PM2.5 - Local Conditions'], df['PM2.5-24h'])
    r2 = r2_score(df['PM2.5 - Local Conditions'], df['PM2.5-24h'])
    
    _min = np.min(df[df.columns[-1]])
    mean = np.mean(df[df.columns[-1]])
    median = np.median(df[df.columns[-1]])
    _max = np.max(df[df.columns[-1]])
    
    result = pd.DataFrame(index = [name],
                          data={'MAE': mae,
                                'MSE': mse,
                                'MdAE': mdae,
                                'R2': r2,
#                                 'MIN': _min,
                                'MEAN': mean,
#                                 'MEDIAN': median,
#                                 'MAX': _max
                               })
    to_concat.append(result)
    
results = pd.concat(to_concat)

In [52]:
results.sort_values('MAE')

,MAE,MSE,MdAE,R2,MEAN
Vermont_Chittenden_7.0,2.337560,9.918968,1.70,-0.167323,3.064665
Wyoming_Fremont_99.0,2.367208,15.283884,2.00,-0.034470,2.484568
Colorado_Rio Blanco_6.0,2.979222,19.740188,2.00,-0.111348,7.810109
Washington_King_80.0,3.112924,22.420511,2.10,-0.189247,6.001201
Vermont_Chittenden_14.0,3.307064,21.417292,2.40,-0.375014,5.744451
New Hampshire_Hillsborough_5001.0,3.347676,20.484188,2.80,-0.259134,5.063079
Maryland_Garrett_2.0,3.528314,21.286269,3.00,-0.330696,6.024054
California_Santa Barbara_2011.0,3.666624,25.462584,3.00,0.086249,8.001924
Vermont_Bennington_4.0,3.836010,25.875054,3.00,-0.316217,5.696943
New Mexico_Bernalillo_23.0,3.922860,39.504635,2.50,-0.336978,6.300803


In [27]:
df['PM2.5 - Local Conditions'].describe()

count    31298.00000
mean        16.13151
std         14.94622
min         -5.00000
25%          7.00000
50%         12.00000
75%         20.00000
max        239.00000
Name: PM2.5 - Local Conditions, dtype: float64

7.749313055147294
137.88727714230942
5.0
0.3827297509054153
